In [9]:
#!/usr/bin/env python

# Copyright (c) 2021- The University of Notre Dame.
# This software is distributed under the GNU General Public License.
# See the file COPYING for details.

# This program is a demonstration of using Work Queue with PyTask
# PyTask allows python functions and their arguments to be bottled up and sent as a task to workers
# This example program finds prime numbers in a range
# PyTask sends the function to find the primality of a given number as a task for all numbers in the given range

# How to work this program:
# Run this cell, and afterwards two sliders and a textbox will appear containing parameters for the program that can be adjusted
# Starting number is the first number to be checked for primality, and the Number of tasks is how many numbers after the starting number that will also be checked
# Max number of workers defines a value for what is the largest amount of workers the program will create
# Keep them at the default values for the first time the program is ran
# Afterwards, run the cell below
# Real time output, as well as some informative diagnostic numbers, will then be shown at the very bottom of the notebook
# Grey cells are tasks that have been submitted, yellow cells are tasks that are currently being worked on,
# Dark green cells are tasks that have finished and returned that the number is composite, light green cells are prime numbers
# After the Workqueue Status textbox displays "All tasks complete!", the program is finished! Then feel free to play around with the numbers below

import work_queue as wq
import ipywidgets as widgets
import time
from widget_control import get_user_input_widgets, get_output_widgets, create_progress_tracker

input_widgets = get_user_input_widgets() # creates the three input widgets below this cell
display(widgets.HBox(input_widgets)) # displays the input widgets 

In [12]:
def is_prime(number): # this is the function to determine the primality of a number
    import math, time # PyTask uses this function to create the tasks in order to determine which numbers are prime
    for i in range(2, int(math.sqrt(number)) + 1):
            if (number % i == 0):
                return 0 # composite
    return 1 # prime

# the three following lines are just shortcuts to access the input widget numbers
number_of_tasks = input_widgets[0] 
number_of_workers = input_widgets[1]
starting_number = input_widgets[2]

# function to create the display at the bottom of the notebook
output_widgets = get_output_widgets(number_of_tasks.value, number_of_workers.value)
status_message = output_widgets[5]
worker_time = output_widgets[6]

# display output information
display(widgets.HBox([status_message, output_widgets[3], worker_time]))
display(widgets.HBox([output_widgets[0], output_widgets[1], output_widgets[2]]))
task_output_storage = []
create_progress_tracker(task_output_storage, number_of_tasks.value, starting_number.value, number_of_workers.value)

# create the Work Queue queue and specify the number of workers as well as the properties of those workers
status_message.value = "Initializing Work Queue"
q = wq.WorkQueue(0) # specify the port to create the workers on. 0 selects a random unused port
workers = wq.Factory('local', manager_host_port=f'localhost:{q.port}')
workers.cores = 1
workers.memory = 100
workers.disk = 100
workers.min_workers = 5
workers.max_workers = number_of_workers.value

# loop through the range of numbers desired and use PyTask to create tasks to determine their primality
status_message.value = "Creating tasks"
for i in range(starting_number.value, starting_number.value + number_of_tasks.value + 1):
    p_task = wq.PythonTask(is_prime, i) # create the task
    p_task.specify_environment("worker-env.tar.gz") # This line ensures that the workers have access to the proper libraries
    q.submit(p_task) # submit the tasks into the queue
    
starting_time = time.perf_counter() # track time since tasks were submitted: used in output display
status_message.value = "Initializing workers"
workers_initialized = False # used to track status messages

with workers: # this line instantiates the workers
    while not q.empty(): # continue until all the tasks have been completed
        if workers_initialized: 
            status_message.value = "Waiting for workers to return"
        t = q.wait(5) # give control to Work Queue so that it can talk to workers
        
        if t:
            workers_initialized = True 
            status_message.value = "Updating output"
            x = t.output
            if x == 1: # if x is a prime number then update it accordingly
                task_output_storage[int(t.id) + 1].disabled = True
            task_output_storage[int(t.id) + 1].button_style = 'success'
            
            for i in range(q.stats.workers_connected + 1): # update the display of which tasks are currently being worked on
                if (int(t.id) + 1 - i) > 0:
                    if (task_output_storage[int(t.id) + 1 - i].button_style == ''):
                        task_output_storage[int(t.id) + 1 - i].button_style='warning'
                        
        # update output widgets
        output_widgets[0].value = q.stats.tasks_done
        output_widgets[1].value = q.stats.workers_connected
        output_widgets[2].value = q.stats.workers_idle
        output_widgets[3].value = output_widgets[0].value / (time.perf_counter() - starting_time)
        worker_time.value = str(q.stats.time_workers_execute / 1000000)
                              
status_message.value = "All tasks complete!"